# Build a Shiny App To Review LLM Data

> How to use `langfree` to build an app you can use to review LLM data.

The motivation for building your own review app is discussed on [the homepage](../#Motivation).  This tutorial walks you through how you can build a minimal app desiged for a single user without authentication.

### Prerequisites

- Runs logged to LangSmith that contain `ChatOpenAI` child runs.
- Set the `LANGCHAIN_API_KEY`, `LANGSMITH_PROJECT_ID` and `LANGCHAIN_ENDPOINT` environment variables as [described here](https://docs.smith.langchain.com/).
- Install langfree: `pip install langfree`
- Clone this repo: `git clone https://github.com/parlance-labs/langfree.git`

### 1. Pull Data From Langsmith

First, we will pull data from LangSmith.  There are many ways to do this, including using the `langsmith` client, which we illustrate below.  

We will pull four specific run ids, parse the data and save it to a dataframe named `sample_data.pkl` that we will use as our backend "database" [^1].  We initialize all records to have a status of `Pending`.

[^1]: We recommend using a real database for production applications, but this allows you to get an understanding of how the system works.

In [ ]:
#|hide
#|eval: false
import os
from langfree.chatrecord import ChatRecordSet             
from langfree.runs import _temp_env_var
from langsmith import Client

_tst_run_ids = ['a05e1668-57b4-4e4d-99d9-1f8578ddba5d',
                '6b9f6c78-dbef-4352-8e4e-0b1777b59cf0',
                'cebad2c1-a00b-43ee-86d0-1d42310e744a',
                '2e1e7686-ae4b-45ab-bae0-0fb18749f1d2']

tmp_env = ({'LANGCHAIN_API_KEY': os.environ['LANGCHAIN_API_KEY_PUB'],
            'LANGSMITH_PROJECT_ID':'2a9996a3-f2d2-4c96-9bea-31a926c18b55'})

with _temp_env_var(tmp_env): # sets temporary enviornment variabes for testing.
    llm_data=ChatRecordSet.from_run_ids(_tst_run_ids)
    llm_data_df = llm_data.to_pandas()
    llm_data_df['status'] = 'Pending'
    llm_data_df.to_pickle('_data/sample_data.pkl')

```python
from langsmith import Client                               # <1>
from langfree.chatrecord import ChatRecordSet              # <2>

# Change these run IDs to your runs
run_ids = ['a05e1668-57b4-4e4d-99d9-1f8578ddba5d',         # <3>
           '6b9f6c78-dbef-4352-8e4e-0b1777b59cf0',         # <3>
           'cebad2c1-a00b-43ee-86d0-1d42310e744a',         # <3>
           '2e1e7686-ae4b-45ab-bae0-0fb18749f1d2']         # <3>

llm_data=ChatRecordSet.from_run_ids(run_ids)               # <4>
llm_data_df = llm_data.to_pandas()                         # <5>
llm_data_df['status'] = 'Pending'                          # <6>
llm_data_df.to_pickle('_data/sample_data.pkl')             # <7>
```
1. The [langsmith](https://github.com/langchain-ai/langsmith-sdk) offers the simplest way to retreive runs.  However, there are additional utilities for retreiving runs provided in [`langfree.runs`](../01_runs.ipynb).
2. `ChatRecordSet` allows you to parse and extract key information from your langchain [ChatOpenAI](https://api.python.langchain.com/en/latest/chat_models/langchain.chat_models.openai.ChatOpenAI.html) runs.
3. These are the run ids that we will pull from langsmith.  You will have to pull your own run ids.  Make sure that your runs have at least one child that is of type `ChatOpenAI` for this to work. 
4. `ChatRecordSet.from_run_ids` allows you to fetch and parse this data froma list of run ids.
5. `ChatRecordSet.to_pandas` allows you to convert this data into a pandas DataFrame
6. The status of each record is initialized to `Pending` which will changed by the front end app depending on user actions.
7. Finally, we save the data to `_data/sample_data.pkl` which will be read by the front end application.

### 2. Run the Shiny App Locally

Assuming you have completed the [Prerequisites](#prerequisites), go to the `nbs/tutorials/` folder from the root of the `langfree` repo you cloned locally. 

Execute Shiny by running:

```
shiny run app.py --reload
```

![](langfree.png)

### 3. Read The Code & Modify It

The web application defined in [app.py](https://github.com/parlance-labs/langfree/blob/main/nbs/tutorials/app.py) is around 150 lines of code, in python.  This means you can easily hack it suit your own needs!  Some resources that are helpful in understanding Shiny and this code:

- Do the excercises on Gordon Shotwell's [Shiny for Python Tutorial](https://posit-dev.github.io/shiny-python-workshop-2023/)
- Read the [Shiny for Python documentation](https://shiny.posit.co/py/docs/overview.html).

#### Suggested modifications:

- Add more information to the side panel.
- Connect to a remote backend database (instead of the DataFrame) using [reactive.poll](https://shiny.posit.co/py/api/reactive.poll.html) or [reactive.file_reader](https://shiny.posit.co/py/api/reactive.file_reader.html).
- Advanced: adapt the hotkeys from this [Wordle example](https://shinylive.io/py/examples/#wordle) so that they work these buttons.

### 4. Deploy

Deploying this application is beyond the scope of this tutorial. However, there are detailed guides for [cloud hosting](https://shiny.posit.co/py/docs/deploy-cloud.html) and [self hosted deployments](https://shiny.posit.co/py/docs/deploy-on-prem.html) for Shiny apps.